In [ ]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher
import requests 
from bs4 import BeautifulSoup
from PyPDF2 import PdfFileReader
import io
#import sys

In [ ]:
request_obj = requests.Session()
url = "http://www.covidmaroc.ma/Pages/LESINFOAR.aspx"
code_soup= request_obj.get(url,headers={'User-Agent': 'Mozilla/5.0'})
soup = BeautifulSoup(code_soup.content , "lxml")

In [ ]:
td_class = soup.find_all("td", class_="ms-rteTableEvenCol-5")
urls=["http://www.covidmaroc.ma"+  td_class[i].find('a').get('href') for i in range(len(td_class))]

In [ ]:
#Noms des régions au Maroc
NState=['Région Grand Casablanca-Settat ',
 'Région Marrakech- Safi ',
 'Région Darâa-Tafilalet ',
 'Région Fès - Meknès ',
 'Région Rabat - Salé-Kenitra ',
 'Région Tanger -Tétouan- Al Hoceima ',
 'Région Dakhla-Oued Eddahab ',
 'Région Souss - Massa ',
 'Région Beni Mellal- Khénifra ',
 "Région de l'Oriental ",
 'Région Laâyoune - Sakia El Hamra ',
 'Région Guelmim Oued Noun ']

In [ ]:
#Noms province 
NProv=['Casablanca',
 'Nouaceur',
 'Mohammadia',
 'ElJadida',
 'Berrechid',
 'Mdiouna',
 'Settat',
 'SidiBennour',
 'Benslimane',
 'Ttouan',
 'Tanger-Assilah',
 'AlHoceima',
 "M'Diq-Fnideq",
 'Ouezzane',
 'Chefchaouen',
 'Larache',
 'FahsAnjra',
 'Knitra',
 'Khmisset',
 'Skhirate-Tmara',
 'SidiKacem',
 'Rabat',
 'Sal',
 'SidiSlimane',
 'Agadir-Ida-Ou-Tanane',
 'Inezgane-AitMelloul',
 'Taroudannt',
 'Tiznit',
 'Tata',
 'Chtouka-AitBaha',
 'Marrakech',
 'Youssoufia',
 'Essaouira',
 'Safi',
 'Rehamna',
 'Chichaoua',
 'ElKeladesSraghna',
 'AlHaouz',
 'Nador',
 'Oujda-Angad',
 'Driouch',
 'Berkane',
 'Guercif',
 'Jerada',
 'Taourirt',
 'Figuig',
 'Errachidia',
 'Ouarzazate',
 'Zagora',
 'Tinghir',
 'Midelt',
 'FquihBenSalah',
 'BniMellal',
 'Azilal',
 'Khnifra',
 'Khouribga',
 'Fs',
 'Mekns',
 'Taounate',
 'Taza',
 'Sefrou',
 'ElHajeb',
 'Boulemane',
 'Ifrane',
 'MoulayYaacoub',
 'Layoune',
 'Es-Semara',
 'Boujdour',
 'Tarfaya',
 'Guelmim',
 'TanTan',
 'SidiIfni',
 'Assa-Zag',
 'OuedEd-Dahab',
 'Aousserd']

In [ ]:
Ncs=pd.DataFrame(index=range(len(urls)),columns=NState,dtype=float)
Nds=pd.DataFrame(index=range(len(urls)),columns=NState,dtype=float)
Ncp=pd.DataFrame(index=range(len(urls)),columns=NProv,dtype=float)
Ndp=pd.DataFrame(index=range(len(urls)),columns=NProv,dtype=float)

In [ ]:
def check(nom):
    l=''
    for cr in nom :
        if cr.isascii():
            l+=cr
    if l[0].islower() :
        return l[1:]
    else :
        return l

In [ ]:
def Keepalpha(x):
    a=''
    for st in x:
        if st.isalpha():
            a+=st
    return a

In [ ]:
def Matcher(region1,region2):
    region2=region2.replace(' ','')
    region1=region1.replace(' ','')
    region2=region2.replace('Région',"")
    region1=region1.replace('Région',"")
    region2=Keepalpha(region2).lower()
    region1=Keepalpha(region1).lower()
    return SequenceMatcher(None,region1,region2).ratio()

In [ ]:
%%time
for IdxUrl in range(len(urls)):
    response=requests.get(urls[IdxUrl],headers={'User-Agent': 'Mozilla/5.0'})
    try :
        with io.BytesIO(response.content) as f:
            pdf = PdfFileReader(f)
            text=[]
            for numpage in range(pdf.getNumPages()):
                page = pdf.getPage(numpage)
                page_content = page.extractText()
                page_content=page_content.replace('\n','')[1:]
                page_content=page_content.split()
                text+=page_content
    except Exception as e:
        print("Oops!", e.__class__, "occurred.")
        print('Fin '+str(IdxUrl)+' : '+urls[IdxUrl])
        print('-'*60)
        continue 
    Region_index=[]
    for index in range(0,len(text)):
        if text[index]=='Région':
            Region_index.append(index)
    Region_index+=[len(text)]
    Regions=[]
    for reg in range(len(Region_index)-1):
        data=dict()
        debart=Region_index[reg]
        fin=Region_index[reg+1]
        indice=debart
        Nom_Region=''
        while text[indice].isnumeric()== False :
            Nom_Region+= text[indice]+' '
            indice+=1
        if text[indice].isnumeric():
            indice+=1
            if  indice < len(text):
                if text[indice].isnumeric() :
                        data[Nom_Region]=[int(text[indice-1]),int(text[indice])]
                else :
                        data[Nom_Region]=[int(text[indice-1]),0]
            else :
                data[Nom_Region]=[int(text[indice-1]),0]
        Nom_prefecture=''
        stat=[]
        prefecture=[]
        stats=[]
        while indice<fin:
            if text[indice].isnumeric()== False :
                Nom_prefecture+= text[indice]
                indice+=1
            elif text[indice].isnumeric()== True and Nom_prefecture!='' :
                prefecture.append(Nom_prefecture)
                Nom_prefecture =''
                stat.append(int(text[indice]))
                indice+=1
                if indice <= fin :
                    if indice==len(text):
                        stat.append(0)
                    elif text[indice].isnumeric():
                        stat.append(int(text[indice]))
                        indice+=1
                    else :
                        stat.append(0)
                    stats.append(stat)
                    stat=[]
            else :
                indice+=1
        for i in range(len(prefecture)):
            data[prefecture[i]]=stats[i]
        Regions.append(data)
    NomRegion=[]
    NomPref=[]
    for dt in Regions:
        prov=list(dt.keys())
        if prov[0].count('Région') <2 and len(dt) >1:
            NomRegion.append(prov[0])
            NomPref+=prov[1:]
        else :
            Regions.remove(dt)
    NomPref=[ check(x) for x in NomPref]
    nvcr=[]
    nvdr=[]
    nvcp=[]
    nvdp=[]
    for dt in Regions:
        donne=list(dt.items())[0]
        nvcr+=[donne[1][0]]
        nvdr+=[donne[1][1]]
        for val in list(dt.items())[1:]:
            nvcp+=[val[1][0]]
            nvdp+=[val[1][1]]
    #Pour chaque région
    for i in range(len(NomRegion)) :
        nom=NomRegion[i]
        if nom=="Région Oriental ":
            nom="Région de l'Oriental "
        if nom in NState :
            Ncs.loc[IdxUrl][nom]=nvcr[i]
            Nds.loc[IdxUrl][nom]=nvdr[i]
        else:
            ratios=[ Matcher(nom,nst) for nst in NState]
            idxmax=np.argmax(np.array(ratios))
            nom_match=NState[idxmax]
            Ncs.loc[IdxUrl][nom_match]=nvcr[i]
            Nds.loc[IdxUrl][nom_match]=nvdr[i]
    #Pour chaque province
    for i in range(len(NomPref)):
        nom=NomPref[i]
        if nom=="MoulayYacoub":
            nom='MoulayYaacoub'
        if nom in NProv :
            Ncp.loc[IdxUrl][nom]=nvcp[i]
            Ndp.loc[IdxUrl][nom]=nvdp[i]
        else:
            ratios=[ Matcher(nom,nprov) for nprov in NProv]
            idxmax=np.argmax(np.array(ratios))
            nom_match=NProv[idxmax]
            Ncp.loc[IdxUrl][nom_match]=nvcp[i]
            Ndp.loc[IdxUrl][nom_match]=nvdp[i]
    print('Fin '+str(IdxUrl)+' : '+urls[IdxUrl])
    print('-'*60)

In [ ]:
Ncs.fillna(0,inplace=True)
Nds.fillna(0,inplace=True)
Ncp.fillna(0,inplace=True)
Ndp.fillna(0,inplace=True)

In [ ]:
date_index=list(pd.date_range('20200903',periods=345))
date_index.reverse()

In [ ]:
Ncs.index=date_index
Nds.index=date_index
Ncp.index=date_index
Ndp.index=date_index

In [ ]:
Ncs.iloc[::-1].to_excel(r'C:/Users/pc/Downloads/NC par régions.xlsx')
Nds.iloc[::-1].to_excel(r'C:/Users/pc/Downloads/ND par régions.xlsx')
Ncp.iloc[::-1].to_excel(r'C:/Users/pc/Downloads/NC par provinces.xlsx')
Ndp.iloc[::-1].to_excel(r'C:/Users/pc/Downloads/ND par provinces.xlsx')